In [1]:
import torch

In [90]:
inputs = torch.tensor(
    [
        [0.43,0.15,0.89],
        [0.55,0.87,0.66],
        [0.57,0.85,0.64],
        [0.22,0.58,0.33],
        [0.77,0.25,0.10],
        [0.05,0.80,0.55]
    ]
)

In [3]:
query = inputs[1]
atten_score_2 = torch.empty(inputs.shape[0])

In [4]:
for i , x_i in enumerate(inputs):
    print(x_i,query)
    atten_score_2[i] = torch.dot(x_i,query)

tensor([0.4300, 0.1500, 0.8900]) tensor([0.5500, 0.8700, 0.6600])
tensor([0.5500, 0.8700, 0.6600]) tensor([0.5500, 0.8700, 0.6600])
tensor([0.5700, 0.8500, 0.6400]) tensor([0.5500, 0.8700, 0.6600])
tensor([0.2200, 0.5800, 0.3300]) tensor([0.5500, 0.8700, 0.6600])
tensor([0.7700, 0.2500, 0.1000]) tensor([0.5500, 0.8700, 0.6600])
tensor([0.0500, 0.8000, 0.5500]) tensor([0.5500, 0.8700, 0.6600])


In [5]:
atten_score_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [6]:
attn_weights_2 = torch.softmax(atten_score_2,dim=0)
attn_weights_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [7]:
context_vec_2 = torch.zeros(query.shape)

In [8]:
for i ,x_i in enumerate(inputs):
    context_vec_2 += x_i * attn_weights_2[i]
print(context_vec_2)


tensor([0.4419, 0.6515, 0.5683])


Now compute attention Score (similrties) for all input

In [9]:
atten_scores = torch.empty(6,6)
for i , x_i in enumerate(inputs):
    for j , x_j in enumerate(inputs):
        atten_scores[i,j] = torch.dot(x_i,x_j)
print(atten_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [10]:
torch.dot(inputs[0],inputs[0])

tensor(0.9995)

Short Cut code to acheive the same faster

In [11]:
atten_scores = inputs @ inputs.T

In [12]:
atten_weights = torch.softmax(atten_scores,dim=1)

In [13]:
atten_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [14]:
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [15]:
atten_weights[3].sum()

tensor(1.)

In [16]:
all_context_vecs = atten_weights @ inputs

In [17]:
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

Self-Attention

In [18]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [19]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)

In [20]:
print(x_2.shape,W_query.shape)

torch.Size([3]) torch.Size([3, 2])


In [21]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

In [22]:
query_2

tensor([0.4306, 1.4551])

get all keys and values one time

In [23]:
keys = inputs @ W_key
values = inputs @ W_value

Compute attention score omega22

In [24]:
keys_2 = keys[1]
atten_score_22 = query_2.dot(keys_2)
print(atten_score_22)

tensor(1.8524)


In [25]:
print(query_2.shape,keys.T.shape)

torch.Size([2]) torch.Size([2, 6])


In [26]:
atten_scores_2 = query_2 @ keys.T
print(atten_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


Compute Attention Score

In [27]:
d_k = keys.shape[-1]
atten_weights_2 = torch.softmax(atten_scores_2/ d_k**0.5,dim=-1)
print(atten_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [28]:
print(atten_weights_2.shape ,values.shape)

torch.Size([6]) torch.Size([6, 2])


In [29]:
context_vec_2 = atten_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


In [30]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self,d_in,d_out):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out
        self.W_query = nn.Parameter(torch.rand(d_in,d_out))
        self.W_key = nn.Parameter(torch.rand(d_in,d_out))
        self.W_value = nn.Parameter(torch.rand(d_in,d_out))
    def forward(self, x):
        keys = x @ self.W_key
        values = x @ self.W_value
        queries = x @ self.W_query
        atten_score = queries @ keys.T
        atten_weights = torch.softmax(atten_score / keys.shape[-1]**0.5,dim=-1)
        context_vec = atten_weights @ values
        return context_vec

In [31]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in,d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [32]:
import torch.nn as nn

class SelfAttention_v2(nn.Module):
    def __init__(self,d_in,d_out,qkv_bias=False):
        super().__init__()
        self.d_in = d_in
        self.d_out = d_out
        self.W_query = nn.Linear(d_in,d_out,bias= qkv_bias)
        self.W_key = nn.Linear(d_in,d_out,bias= qkv_bias)
        self.W_value = nn.Linear(d_in,d_out,bias= qkv_bias)
    def forward(self, x):
        keys = self.W_key(x)
        values = self.W_value(x)
        queries = self.W_query(x)
        atten_score = queries @ keys.T
        atten_weights = torch.softmax(atten_score / keys.shape[-1]**0.5,dim=-1)
        context_vec = atten_weights @ values
        return context_vec

In [33]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


# Exercise 3.1:what is the difference between v1 and v2: is linear layer is more better in matrix multiplication and do it automaticly


In [34]:
sa_v1.W_query.data  = sa_v2.W_query.weight.T
sa_v1.W_value.data = sa_v2.W_value.weight.T
sa_v1.W_key.data = sa_v2.W_key.weight.T


In [35]:
print(sa_v1(inputs),"\n",sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>) 
 tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


# End of the Exercise

Casual Attention Saga xD

In [36]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)

In [37]:
atten_scores = queries @ keys.T
atten_weights = torch.softmax(atten_scores/keys.shape[-1]**0.5,dim=-1)

In [38]:
print(atten_weights)


tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [39]:
print(atten_weights.shape)

torch.Size([6, 6])


In [40]:
context_length = atten_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length,context_length))

In [41]:
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [42]:
masked_simple = atten_weights * mask_simple

In [43]:
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [44]:
rows_sum = masked_simple.sum(dim=1,keepdim=True)
print(rows_sum)

tensor([[0.1921],
        [0.3700],
        [0.5357],
        [0.6775],
        [0.8415],
        [1.0000]], grad_fn=<SumBackward1>)


In [45]:
masked_simple_norm = masked_simple/rows_sum
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


Trick to compute More effecient

In [46]:
mask = torch.triu(torch.ones(context_length,context_length),diagonal=1)

In [47]:
mask.bool()

tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])

In [48]:
masked = atten_scores.masked_fill(mask.bool(),-torch.inf)

In [49]:
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [50]:
atten_weights = torch.softmax(masked/keys.shape[-1]**0.5,dim=-1)

In [51]:

print(atten_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [52]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6,6)

In [53]:
print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [54]:
torch.manual_seed(123)
print(dropout(atten_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


In [91]:
batch = torch.stack((inputs,inputs),dim=0)
batch.shape

torch.Size([2, 6, 3])

In [56]:
batch

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])

In [57]:
class CausalAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,drop_out,qkv_bias=False):
        super().__init__()
        self.d_out=d_out
        self.W_query = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.dropout = torch.nn.Dropout(drop_out)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))
    def forward(self,x):
        b,num_tokens,d_in = x.shape #(batch_size,number of words entered ,embedding size)
        quries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        atten_scores = quries@keys.transpose(1,2)
        atten_scores.masked_fill(self.mask.bool()[:num_tokens,:num_tokens],-torch.inf)
        atten_weights = torch.softmax(atten_scores / keys.shape[-1]**0.5,dim=-1)
        atten_weights = self.dropout(atten_weights)

        context_vec = atten_weights@values
        return context_vec

In [ ]:
torch.manual_seed(123)
context_length = batch.shape[1] #num of words
ca = CausalAttention(d_in,d_out,context_length,0.0)
context_vecs = ca(batch)

In [ ]:
context_vecs.shape

torch.Size([2, 6, 2])

Multi_Head Attention:

In [61]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self,d_in,d_out,context_length,drop_out,num_heads,qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
           [ CausalAttention(d_in,d_out,context_length,drop_out,qkv_bias)for _ in range(num_heads)]
        )
    def forward(self,x):
        return torch.cat([head(x) for head in self.heads],dim=-1)

In [62]:
torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)


In [63]:
print(context_vecs)

tensor([[[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.5337, -0.1051,  0.5085,  0.3508],
         [-0.5323, -0.1080,  0.5084,  0.3508],
         [-0.5323, -0.1079,  0.5084,  0.3506],
         [-0.5297, -0.1076,  0.5074,  0.3471],
         [-0.5311, -0.1066,  0.5076,  0.3446],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)


In [65]:
print(context_vecs.shape)

torch.Size([2, 6, 4])


In [97]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, \
            "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )
    def forward(self,x):
        b,num_tokens,d_in = x.shape

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1,2)
        queries = queries.transpose(1,2)
        values = values.transpose(1,2)


        atten_score = queries @ keys.transpose(2,3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        atten_score.masked_fill_(mask_bool, -torch.inf)

        atten_weights = torch.softmax(atten_score/keys.shape[-1]**0.5,dim=-1)
        atten_weights = self.dropout(atten_weights)

        context_vec = (atten_weights @ values).transpose(1,2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vec





In [98]:
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


In [71]:
t=t.view(2,6,2,1)

In [ ]:
t

In [76]:
t.transpose(1,2)

tensor([[[[ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 1.4013e-45],
          [ 0.0000e+00]],

         [[ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00]]],


        [[[ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 1.1210e-44],
          [ 1.4013e-45],
          [-1.7014e+38]],

         [[ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 1.1515e-40]]]])

In [77]:
torch.inf

inf